### Import

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import  OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier

### Data Load

In [2]:
train = pd.read_csv('./Data/train.csv').drop(columns=['ID'])
test = pd.read_csv('./Data/test.csv').drop(columns=['ID'])

In [3]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

### Data Pre-processing

In [4]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

In [5]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

In [6]:
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [7]:
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

In [8]:
X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(0)
X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(0)

In [9]:
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif

# 1️⃣ 상수 Feature 제거
var_thresh = VarianceThreshold(threshold=0.0)
X_train_var_filtered = var_thresh.fit_transform(X_train_encoded)

# 2️⃣ Feature Selection 적용
selector = SelectKBest(f_classif, k=20)
X_train_selected = selector.fit_transform(X_train_var_filtered, y)

# 3️⃣ 언더샘플링 적용 (Feature Selection 후)
undersampler = RandomUnderSampler(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X_train_selected, y)

### Train

SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score

# 데이터 스케일링 (SVM은 특성 크기에 민감)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)

# SVM 모델 초기화 (불필요한 probability 제거)
svm_model = SVC(kernel='rbf', C=1.0, probability=True, random_state=42)

# 모델 학습
# svm_model.fit(X_train_scaled, y.ravel())
svm_model.fit(X_resampled, y_resampled.ravel())


# # 예측
# y_train_pred = svm_model.predict(X_train_scaled)
# y_train_proba = svm_model.decision_function(X_train_scaled)  # decision_function 활용

# # 평가
# accuracy = accuracy_score(y, y_train_pred)
# roc_auc = roc_auc_score(y, y_train_proba)  # decision_function 사용 시 더 안정적

# # 출력
# print(f"SVM Model - Accuracy: {accuracy:.4f}")
# print(f"SVM Model - ROC-AUC Score: {roc_auc:.4f}")

C:\Users\vhehr\AppData\Local\Temp\ipykernel_23620\3835645471.py:14: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  svm_model.fit(X_resampled, y_resampled.ravel())


In [12]:

# 예측
y_train_pred = svm_model.predict(X_resampled)
y_train_proba = svm_model.decision_function(X_resampled)  # decision_function 활용

# 평가
accuracy = accuracy_score(y_resampled, y_train_pred)
roc_auc = roc_auc_score(y_resampled, y_train_proba)  # decision_function 사용 시 더 안정적

# 출력
print(f"SVM Model - Accuracy: {accuracy:.4f}")
print(f"SVM Model - ROC-AUC Score: {roc_auc:.4f}")

KeyboardInterrupt: 

### Predict

In [13]:
pred_proba = svm_model.predict_proba(X_test_encoded)[:, 1]

AttributeError: This 'SVC' has no attribute 'predict_proba'

### Submission

In [26]:
sample_submission = pd.read_csv('./Data/sample_submission.csv')
sample_submission['probability'] = pred_proba

In [28]:
sample_submission.to_csv('./RandomForest_RandomSearch_submit.csv', index=False)